In [ ]:
from Utils import *
import cv2


In [ ]:
image_path = "data/images/greenspillexample.jpg"
class_indices = [2, 3]
seg_img = segment_face_et_al(og_img_path=image_path, class_indices=class_indices)
# NOTE: bin_mask and seg_img are numpy arrays

In [ ]:
hls_seg = cv2.cvtColor(seg_img, cv2.COLOR_BGR2HLS)
seg_hue = hls_seg[:, :, 0]
create_colour_heatmap(image_hue=seg_hue)

In [ ]:
save_img_as(fname="outputs/example_seg_img.jpg", img=seg_img)